<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/Multi_Agent_Solution_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Core LangChain packages
!pip install langchain langchain-community -q

# LLM integrations for OpenAI and xAI
!pip install langchain-openai -q

# NOTE: The package for xAI/Grok integration in LangChain is called 'langchain-xai'.
# Install this package to use Grok models.
!pip install langchain-xai -q

# Search tool for the Research Agent
!pip install duckduckgo-search -q

In [ ]:
!pip uninstall duckduckgo-search -y
!pip install ddgs

In [3]:
import os
from langchain.agents import initialize_agent, AgentType
from langchain_openai import ChatOpenAI
from langchain.tools import Tool
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain_xai import ChatXAI
from ddgs import DDGS  # Import the DDGS class

import warnings
warnings.filterwarnings("ignore")

# --- 1. Setup and LLM Configuration ---
# Configuration for API keys
XAI_API_KEY = None
OPENAI_API_KEY = None

try:
    from google.colab import userdata
    XAI_API_KEY = userdata.get('XAI_KEY')
    if not XAI_API_KEY:
        print("WARNING: XAI_KEY not found in Colab secrets.")

    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    if not OPENAI_API_KEY:
        print("WARNING: OPENAI_API_KEY not found in Colab secrets.")

except ImportError:
    print("WARNING: Not in Colab. Attempting to get API keys from environment variables.")
    XAI_API_KEY = os.environ.get('XAI_KEY', None)
    OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY', None)

# Define the LLMs to use
llms_to_use = {}

if OPENAI_API_KEY:
    # Research Agent uses the GPT-4o-mini model, which supports the 'stop' argument.
    llms_to_use['research_llm'] = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=0.5)
else:
    print("FATAL ERROR: OPENAI_API_KEY not set. The research agent cannot be initialized.")
    exit()

if XAI_API_KEY:
    # Writing Agent uses the Grok-4 model for content generation.
    llms_to_use['writer_llm'] = ChatXAI(model="grok-4", xai_api_key=XAI_API_KEY, temperature=0.7)
else:
    print("WARNING: XAI_API_KEY not set. Only using GPT for the writing agent as a fallback.")
    # Fallback to OpenAI if Grok key is missing
    llms_to_use['writer_llm'] = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=0.7)

print("LLMs configured successfully.")

# --- 2. Define Agents and Tools ---
print("\nDefining agents and tools...")

# Tool for the Research Agent using DDGS
def ddgs_search(query: str) -> str:
    """Runs a search using DDGS and returns the results."""
    with DDGS() as ddgs:
        results = [r['body'] for r in ddgs.text(query, max_results=5)]
        return "\n".join(results)

search_tool = Tool(
    name="DuckDuckGoSearch",
    description="A search tool that uses DuckDuckGo via the ddgs library. Use this for general web searches.",
    func=ddgs_search
)


# Research Agent: Uses GPT-4o-mini to research a topic
research_agent = initialize_agent(
    tools=[search_tool],
    llm=llms_to_use['research_llm'],
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=False,
    handle_parsing_errors=True
)

# Writing Agent: Uses Grok-4 to write based on research
writer_prompt_template = PromptTemplate(
    input_variables=["research_result"],
    template="You are an expert article writer. Your task is to write a concise, professional article based on the following research data. The article should be a maximum of 3 paragraphs.\n\nResearch Data:\n{research_result}\n\nArticle:"
)
writer_chain = LLMChain(llm=llms_to_use['writer_llm'], prompt=writer_prompt_template)
print("Agents defined successfully.")

# --- 3. Orchestrate the Workflow ---
print("\nStarting the multi-agent workflow...")

# Create the Sequential Chain
full_chain = SimpleSequentialChain(chains=[
    research_agent,
    writer_chain
], verbose=False)

# Run the workflow with a user query
user_query = "What are the key differences between the Grok 4 and GPT-5 models?"
final_article = full_chain.run(user_query)

print("\n--- Final Article Output ---")
print(final_article)

LLMs configured successfully.

Defining agents and tools...
Agents defined successfully.

Starting the multi-agent workflow...

--- Final Article Output ---
### Comparing Grok 4 and GPT-5: Key Differences in AI Capabilities

In the rapidly evolving landscape of artificial intelligence, Grok 4 and GPT-5 stand out as advanced language models developed by competing tech giants. Grok 4, created by xAI, focuses on integrating real-time data processing with a commitment to objectivity, setting it apart in scenarios requiring up-to-the-minute information. Meanwhile, GPT-5, from OpenAI, builds on its predecessors by enhancing core functionalities like reasoning and efficiency. Understanding their differences is crucial for users selecting the right tool for tasks ranging from research to creative generation.

Grok 4 excels in real-time search capabilities, allowing it to pull and synthesize current web data seamlessly, which is ideal for dynamic environments like market analysis or news aggreg